# 실행가능성 확인하기


In [0]:
import glob, os, cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터


In [0]:
EPOCHS = 10

DATASET_PATH = '/content/drive/My Drive/data/fabric/dataset/2/'
DATASET_OK_PATTERN   = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH     = '/content/drive/My Drive/data/fabric/result'

## 단순 모델 설정


In [0]:
def Model():
  return Sequential([ Conv2D( 32, (3,3), activation='relu' ), MaxPool2D(),
                      Conv2D( 64, (3,3), activation='relu' ), MaxPool2D(),
                      Conv2D( 128, (3,3), activation='relu' ), MaxPool2D(),
                      Conv2D( 256, (3,3), activation='relu' ), MaxPool2D(),
                      Flatten(),
                      Dense( 1, activation='sigmoid' )
                     ])

## 데이터셋 불러오기


In [0]:
def preprocess( file_name ):
  img = tf.io.read_file( file_name )
  img = tf.image.decode_png( img )
  return tf.image.convert_image_dtype(img, tf.float32)

In [36]:
ok_list     = glob.glob(DATASET_OK_PATTERN)
ds_ok       = tf.data.Dataset.list_files(ok_list)
sd_ok_label = tf.data.Dataset.from_tensor_slices( [0]*len(ok_list) )

ds_ok       = ds_ok.map(preprocess)
ds_ok       = tf.data.Dataset.zip( (ds_ok, ds_ok_label) )

InvalidArgumentError: ignored

In [0]:
fail_list     = glob.glob(DATASET_FAIL_PATTERN)
ds_fail       = tf.data.Dataset.list_files(fail_list)
sd_fail_label = tf.data.Dataset.from_tensor_slices( [1]*len(fail_list) )

ds_fail       = ds_fail.map(preprocess)
ds_fail       = tf.data.Dataset.zip( (ds_fail, ds_fail_label) )

In [0]:
ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기


In [23]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

NameError: ignored

## 모델 생성 및 학습


In [0]:
model = Model()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

## 결과를 이미지로 저장

In [0]:
def mkdir(path):
  if os.path.exists(path) is False:
    os.mkdir(path)

In [0]:
mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH+'/TN')
mkdir(RESULT_SAVE_PATH+'/TP')
mkdir(RESULT_SAVE_PATH+'/FN')
mkdir(RESULT_SAVE_PATH+'/FP')


In [0]:
index = 0
for imgs, labels in ds_valid:
  preds = model.predict(imgs)
  for idx in range( imgs.shape[0] ):
    gt = labels[idx].numpy()
    y  = preds[idx]

    if gt == 1 and y > 0.5:
      path = RESULT_SAVE_PATH + '/TP'

    if gt == 1 and y <= 0.5:
      path = RESULT_SAVE_PATH + '/FN'

    if gt == 0 and y > 0.5:
      path = RESULT_SAVE_PATH + '/FP'

    if gt == 0 and y <= 0.5:
      path = RESULT_SAVE_PATH + '/TN'
    cv2.imwrite( path + '%.4f_%04d.png'% (y, index), imgs[idx].numpy() * 255)
    index += 1